In [ ]:
#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))  
import IPython.display as ipd

In [ ]:
import os
import gc
import re
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Layer, Input, Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping 
from tensorflow.keras import mixed_precision

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning, module='keras.src.saving.saving_lib')

In [ ]:
# Set up mixed precision policy
mixed_precision.set_global_policy('mixed_float16')

### Multi Sub-directory Processing with Accuracy

In [ ]:
def process_audio_file(file_path, max_length=332):
    # Load the audio file with librosa, handle both mp3 and wav formats
    signal, sr = librosa.load(file_path, sr=16000)
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13, n_fft=256, hop_length=160, n_mels=32, fmin=0, fmax=8000)
    pad_width = max_length - mfccs.shape[1]
    if pad_width > 0:
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfccs

def predict_audio_file(model, label_encoder, mfccs_padded):
    mfccs_padded = mfccs_padded[np.newaxis, ...]  # Add batch dimension
    softmax_output = model.predict(mfccs_padded)[0]  # Get the first batch item
    labels = label_encoder.classes_
    probabilities_df = pd.DataFrame(softmax_output, index=labels, columns=['Probability'])
    return probabilities_df

def process_directory(directory, model, label_encoder):
    results = []
    for root, dirs, files in os.walk(directory):
        for sub_dir in dirs:
            correct_predictions = 0
            total_files = 0
            sub_dir_path = os.path.join(root, sub_dir)
            print(f"Processing subdirectory: {sub_dir_path}")
            for file in tqdm(os.listdir(sub_dir_path), desc=f"Analyzing {sub_dir}"):
                if file.endswith(('.wav', '.mp3')):
                    file_path = os.path.join(sub_dir_path, file)
                    label_from_filename = file.split('_')[0]  # Assuming the label is the first word in the filename
                    mfccs_padded = process_audio_file(file_path)
                    probabilities_df = predict_audio_file(model, label_encoder, mfccs_padded)
                    predicted_label = probabilities_df['Probability'].idxmax()
                    highest_probability = probabilities_df['Probability'].max()
                    results.append([sub_dir, file, label_from_filename, predicted_label, highest_probability])
                    if predicted_label == label_from_filename:
                        correct_predictions += 1
                    total_files += 1
            
            accuracy = (correct_predictions / total_files) * 100 if total_files > 0 else 0
            results.append([sub_dir, "Accuracy", "", "", accuracy])

    results_df = pd.DataFrame(results, columns=['Subdirectory', 'Filename', 'Correct Label', 'Predicted Label', 'Probability'])
    return results_df

# Load the pre-trained model and label encoder
model = load_model('saved_data/models/non-masked_mobilenetv3-finetuned/mobilenetv3small_finetuned_model.keras')
all_labels = ['battery', 'description', 'environment', 'greeting', 'health', 'noise', 'nutrition', 'silence', 'sun', 'water']
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Specify the directory containing subdirectories with audio files
directory = '/Users/ciprian/Desktop/Projects/Smart Plant Pot/Audio/Voice Recognition/Testing Samples'

# Process the directory and get predictions along with accuracy
predictions_df = process_directory(directory, model, label_encoder)
predictions_df

In [ ]:
# Define GradientReversalLayer for adversarial models (if any exist)
@tf.keras.utils.register_keras_serializable()
class GradientReversalLayer(Layer):
    def __init__(self, lambda_=1.0, **kwargs):
        super(GradientReversalLayer, self).__init__(**kwargs)
        self.lambda_ = lambda_

    @tf.custom_gradient
    def call(self, x):
        def grad(dy):
            return -self.lambda_ * dy
        return x, grad

    def get_config(self):
        config = super().get_config()
        config.update({"lambda_": self.lambda_})
        return config

def process_audio_file(file_path, max_length=332):
    # Load the audio file with librosa, handle both mp3 and wav formats
    signal, sr = librosa.load(file_path, sr=16000)
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13, n_fft=256, hop_length=160, n_mels=32, fmin=0, fmax=8000)
    pad_width = max_length - mfccs.shape[1]
    if pad_width > 0:
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfccs

def evaluate_model_directory(model_path, test_directory, label_encoder):
    # Function to evaluate a model using audio files in the test directory
    try:
        custom_objects = {"GradientReversalLayer": GradientReversalLayer}
        model = load_model(model_path, custom_objects=custom_objects)
    except:
        model = load_model(model_path)  # For models without adversarial training
    
    results = []
    y_true, y_pred = [], []
    
    for root, _, files in os.walk(test_directory):
        for file in tqdm(files, desc=f"Processing {os.path.basename(model_path)}"):
            if file.endswith(('.wav', '.mp3')):
                file_path = os.path.join(root, file)
                label_from_filename = file.split('_')[0]  # Assuming the label is the first word in the filename
                mfccs_padded = process_audio_file(file_path)
                mfccs_padded = mfccs_padded[np.newaxis, ...]  # Add batch dimension
                
                predictions = model.predict(mfccs_padded)
                
                if isinstance(predictions, list):  # Adversarial model
                    y_pred_task = predictions[0]
                else:  # Non-adversarial model
                    y_pred_task = predictions
                
                predicted_label = label_encoder.inverse_transform(np.argmax(y_pred_task, axis=1))[0]
                
                y_true.append(label_from_filename)
                y_pred.append(predicted_label)
                
                results.append([file, label_from_filename, predicted_label])
    
    command_accuracy = accuracy_score(y_true, y_pred) * 100
    command_precision = precision_score(y_true, y_pred, average='weighted', zero_division=0) * 100
    command_recall = recall_score(y_true, y_pred, average='weighted', zero_division=0) * 100
    command_f1 = f1_score(y_true, y_pred, average='weighted') * 100
    
    return results, command_accuracy, command_precision, command_recall, command_f1

# Set the path to the models directory and the test data directory
models_directory = 'saved_data/models'
test_directory = '/Users/ciprian/Desktop/Projects/Smart Plant Pot/Audio/Voice Recognition/Testing Samples'

# Label encoder setup
all_labels = ['battery', 'description', 'environment', 'greeting', 'health', 'noise', 'nutrition', 'silence', 'sun', 'water']
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Iterate through subdirectories in the test directory and evaluate models
overall_results = []
subdir_results = {}

for subdir, _, _ in os.walk(test_directory):
    subdir_name = os.path.basename(subdir)
    if subdir_name not in subdir_results:
        subdir_results[subdir_name] = []
    
    for model_subdir, _, model_files in os.walk(models_directory):
        for model_file in model_files:
            if model_file.endswith('.keras'):
                model_path = os.path.join(model_subdir, model_file)
                model_folder = os.path.basename(model_subdir)
                
                model_results, command_accuracy, command_precision, command_recall, command_f1 = evaluate_model_directory(model_path, subdir, label_encoder)
                
                result_entry = {
                    'Model': model_folder,
                    'Command Accuracy (%)': f"{command_accuracy:.2f}",
                    'Command Precision (%)': f"{command_precision:.2f}",
                    'Command Recall (%)': f"{command_recall:.2f}",
                    'Command F1 (%)': f"{command_f1:.2f}"
                }
                subdir_results[subdir_name].append(result_entry)
                overall_results.append(result_entry)

# Display performance DataFrames for each subdirectory
for subdir_name, results in subdir_results.items():
    if results:  # Only display if there are results
        print(f"Performance for {subdir_name}:")
        subdir_df = pd.DataFrame(results)
        ipd.display(ipd.HTML(subdir_df.to_html(index=False)))

# Create a summary DataFrame with the best model for each subdirectory
summary_results = []

for subdir_name, results in subdir_results.items():
    subdir_df = pd.DataFrame(results)
    if not subdir_df.empty and 'Command Accuracy (%)' in subdir_df.columns and 'Command F1 (%)' in subdir_df.columns:
        best_accuracy_model = subdir_df.loc[subdir_df['Command Accuracy (%)'].astype(float).idxmax()]
        best_f1_model = subdir_df.loc[subdir_df['Command F1 (%)'].astype(float).idxmax()]
        summary_results.append({
            'Subdirectory': subdir_name,
            'Best Accuracy Model': best_accuracy_model['Model'],
            'Best Accuracy (%)': best_accuracy_model['Command Accuracy (%)'],
            'Best F1 Model': best_f1_model['Model'],
            'Best F1 Score (%)': best_f1_model['Command F1 (%)']
        })

summary_df = pd.DataFrame(summary_results)

# Display the final summary DataFrame
print("Overall Performance Summary:")
ipd.display(ipd.HTML(summary_df.to_html(index=False)))